In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#We need to do a one-time save all the models to a csv thing.
"""
%run MODEL_LIST.ipynb            #Load the models

model_names = ['model_1', 'model_2', 'model_3', 'model_4', 'model_5', 
               'model_6']

model_models = [model_1, model_2, model_3, model_4, model_5, model_6]

model_evs = [min_ev_1, min_ev_2, min_ev_3, min_ev_4, min_ev_5, min_ev_6]

model_features = [features_1, features_2, features_3, features_4,
                 features_5, features_6]

model_scores = [score_1, score_2, score_3, score_4, score_5, 0]

models = [model_names, model_models, model_features, model_evs, model_scores]

"""

"\n%run MODEL_LIST.ipynb            #Load the models\n\nmodel_names = ['model_1', 'model_2', 'model_3', 'model_4', 'model_5', \n               'model_6']\n\nmodel_models = [model_1, model_2, model_3, model_4, model_5, model_6]\n\nmodel_evs = [min_ev_1, min_ev_2, min_ev_3, min_ev_4, min_ev_5, min_ev_6]\n\nmodel_features = [features_1, features_2, features_3, features_4,\n                 features_5, features_6]\n\nmodel_scores = [score_1, score_2, score_3, score_4, score_5, 0]\n\nmodels = [model_names, model_models, model_features, model_evs, model_scores]\n\n"

In [3]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 2

5


In [4]:
display(models)

[['model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=10000,\n                   multi_class='auto', n_jobs=None, penalty='none',\n                   random_state=75, solver='saga', tol=0.0001, verbose=0,\n                   warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',\n                       max_depth=12, max_features=None, max_leaf_nodes=144,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=2, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='random')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_features='auto',\n  

In [5]:
df=pd.read_csv("data/owl-with-odds.csv")

In [6]:
#Split the test set.  We are always(?) going to use the last 60 matches as the test set, so we don't want those around
#as we pick models
subset = ['t1_odds', 't2_odds']
df.dropna(subset=subset ,inplace=True)
df['t1_odds'] = pd.to_numeric(df['t1_odds'], errors='coerce')
df['t2_odds'] = pd.to_numeric(df['t2_odds'], errors='coerce')
df.dropna(subset=subset ,inplace=True)
#Create the odds df
odds_df = df[subset]

#Create a label df
label_df = df['winner_label']


df_train = df[:-60]
odds_train = odds_df[:-60]
label_train = label_df[:-60]

df_test = df[-60:]
odds_test = odds_df[-60:]
label_test = label_df[-60:]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

60
60
60
534
534
534


In [7]:
display(df_train)
display(df_test)

,Unnamed: 0,Unnamed: 0.1,id,team_one,team_two,stage,winner,date,corona_virus_isolation,t1_wins_season,...,t2_win_percent_last_20,t1_place_last_season,t2_place_last_season,t1_wins_vs_t2,t1_losses_vs_t2,t1_matches_vs_t2,t1_win_percent_vs_t2,winner_label,t1_odds,t2_odds
1,1,1,10224,Los Angeles Gladiators,Shanghai Dragons,Overwatch League - Stage 1,Los Angeles Gladiators,1/11/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,0,-370.0,276.0
2,2,2,10225,Seoul Dynasty,Dallas Fuel,Overwatch League - Stage 1,Seoul Dynasty,1/11/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,0,-588.0,413.0
3,3,3,10226,Florida Mayhem,London Spitfire,Overwatch League - Stage 1,London Spitfire,1/11/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,1,652.0,-1667.0
4,4,4,10227,Houston Outlaws,Philadelphia Fusion,Overwatch League - Stage 1,Philadelphia Fusion,1/11/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,1,-161.0,143.0
5,5,5,10228,New York Excelsior,Boston Uprising,Overwatch League - Stage 1,New York Excelsior,1/12/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,0,-769.0,380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,644,644,32856,Florida Mayhem,Paris Eternal,OWL 2020 Regular Season,Florida Mayhem,4/17/2020,True,2.0,...,0.45,20.0,14.0,0,2,2,0.000000,0,250.0,-345.0
645,645,645,32855,Boston Uprising,Toronto Defiant,OWL 2020 Regular Season,Toronto Defiant,4/18/2020,True,1.0,...,0.20,19.0,17.0,1,2,3,0.333333,1,200.0,-278.0
646,646,646,32853,Dallas Fuel,San Francisco Shock,OWL 2020 Regular Season,San Francisco Shock,4/18/2020,True,2.0,...,0.80,15.0,3.0,1,6,7,0.142857,1,586.0,-1111.0
647,647,647,32851,Chengdu Hunters,Guangzhou Charge,OWL 2020 Regular Season,Chengdu Hunters,4/18/2020,True,2.0,...,0.65,12.0,9.0,1,3,4,0.250000,0,147.0,-196.0


,Unnamed: 0,Unnamed: 0.1,id,team_one,team_two,stage,winner,date,corona_virus_isolation,t1_wins_season,...,t2_win_percent_last_20,t1_place_last_season,t2_place_last_season,t1_wins_vs_t2,t1_losses_vs_t2,t1_matches_vs_t2,t1_win_percent_vs_t2,winner_label,t1_odds,t2_odds
649,649,649,32844,Guangzhou Charge,Hangzhou Spark,OWL 2020 Regular Season,Guangzhou Charge,4/19/2020,True,3.0,...,0.55,9.0,4.0,2,2,4,0.500000,0,100.0,-132.0
650,650,650,32843,Chengdu Hunters,Shanghai Dragons,OWL 2020 Regular Season,Shanghai Dragons,4/19/2020,True,3.0,...,0.55,12.0,11.0,1,3,4,0.250000,1,277.0,-400.0
651,651,651,32842,Seoul Dynasty,Hangzhou Spark,OWL 2020 Regular Season,Seoul Dynasty,4/25/2020,True,2.0,...,0.50,8.0,4.0,0,3,3,0.000000,0,-405.0,360.0
652,652,652,32831,Guangzhou Charge,Shanghai Dragons,OWL 2020 Regular Season,Shanghai Dragons,4/25/2020,True,4.0,...,0.60,9.0,11.0,1,3,4,0.250000,1,400.0,-500.0
653,653,653,32829,New York Excelsior,Chengdu Hunters,OWL 2020 Regular Season,New York Excelsior,4/25/2020,True,5.0,...,0.45,2.0,12.0,0,1,1,0.000000,0,-330.0,270.0
654,654,654,32828,Atlanta Reign,Philadelphia Fusion,OWL 2020 Regular Season,Philadelphia Fusion,4/25/2020,True,4.0,...,0.65,6.0,10.0,0,4,4,0.000000,1,175.0,-210.0
655,655,655,32827,San Francisco Shock,Los Angeles Valiant,OWL 2020 Regular Season,San Francisco Shock,4/25/2020,True,4.0,...,0.50,3.0,13.0,4,5,9,0.444444,0,-680.0,490.0
656,656,656,32826,Hangzhou Spark,Chengdu Hunters,OWL 2020 Regular Season,Hangzhou Spark,4/26/2020,True,3.0,...,0.45,4.0,12.0,4,0,4,1.000000,0,-190.0,165.0
657,657,657,32825,Shanghai Dragons,Seoul Dynasty,OWL 2020 Regular Season,Shanghai Dragons,4/26/2020,True,7.0,...,0.55,11.0,8.0,1,5,6,0.166667,0,-130.0,100.0
658,658,658,32824,Guangzhou Charge,New York Excelsior,OWL 2020 Regular Season,New York Excelsior,4/26/2020,True,4.0,...,0.60,9.0,2.0,1,0,1,1.000000,1,240.0,-360.0


In [8]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])



In [9]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [10]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [11]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)

model_3
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=75, verbose=0,
                       warm_start=False)
['t1_wins_vs_t2', 't2_wins_last_5', 't1_wins_last_10']
4


In [12]:
#Full list of possible features
#Basically inclusive features with the winner_label removed
my_pos_features = ['corona_virus_isolation', 't1_wins_season',
       't1_losses_season', 't2_wins_season', 't2_losses_season',
       't1_matches_season', 't2_matches_season', 't1_win_percent_season',
       't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime',
       't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime',
       't2_matches_alltime', 't1_win_percent_alltime',
       't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3',
       't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3',
       't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5',
       't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5',
       't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10',
       't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10',
       't2_win_percent_last_10', 
        't1_wins_vs_t2', 't1_losses_vs_t2',
       't1_matches_vs_t2', 't1_odds', 't2_odds']

In [ ]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features_v2(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    
    
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)
    
    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    
    keep_going = False
    
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model

    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
        
        
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    
    

Current best score is: 8685.639355703173
Feature: corona_virus_isolation Score: 9346.710070436899
The best feature was corona_virus_isolation.  It scored 9346.710070436899
Current best score is: 9346.710070436899
Feature: t1_matches_season Score: 9740.23322944033
The best feature was t1_matches_season.  It scored 9740.23322944033
Current best score is: 9740.23322944033
Feature: t2_losses_last_5 Score: 10891.231019543857
The best feature was t2_losses_last_5.  It scored 10891.231019543857
Current best score is: 10891.231019543857
NO IMPROVEMENT
FINAL BEST SCORE: 10891.231019543857


Starting New Run for RandomForestClassifier


Previous Best Score: 10891.231019543857
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 10891.231019543857
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 10496.103553785724
Criterion: gini max_features: log2 max_depth: None min_samples_lea

Criterion: entropy max_features: None max_depth: None min_samples_leaf: 1 max_leaf_nodes: 214 n_estimators 100 7772.684563692881
Criterion: gini max_features: auto max_depth: 28 min_samples_leaf: 1 max_leaf_nodes: 214 n_estimators 100 10192.10557096451
Criterion: entropy max_features: auto max_depth: 28 min_samples_leaf: 1 max_leaf_nodes: 214 n_estimators 100 9354.11236357924
Criterion: gini max_features: log2 max_depth: 28 min_samples_leaf: 1 max_leaf_nodes: 214 n_estimators 100 10192.10557096451
Criterion: entropy max_features: log2 max_depth: 28 min_samples_leaf: 1 max_leaf_nodes: 214 n_estimators 100 9354.11236357924
Criterion: gini max_features: None max_depth: 28 min_samples_leaf: 1 max_leaf_nodes: 214 n_estimators 100 7930.85118829262
Criterion: entropy max_features: None max_depth: 28 min_samples_leaf: 1 max_leaf_nodes: 214 n_estimators 100 7772.684563692881
Criterion: gini max_features: auto max_depth: 81 min_samples_leaf: 1 max_leaf_nodes: 214 n_estimators 100 10192.105570964

Criterion: gini max_features: None max_depth: 81 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 7930.85118829262
Criterion: entropy max_features: None max_depth: 81 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 7772.684563692881
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 3 max_leaf_nodes: 925 n_estimators 100 7634.216180923145
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 3 max_leaf_nodes: 925 n_estimators 100 6115.015006081867
Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 3 max_leaf_nodes: 925 n_estimators 100 7634.216180923145
Criterion: entropy max_features: log2 max_depth: None min_samples_leaf: 3 max_leaf_nodes: 925 n_estimators 100 6115.015006081867
Criterion: gini max_features: None max_depth: None min_samples_leaf: 3 max_leaf_nodes: 925 n_estimators 100 8559.984412361047
Criterion: entropy max_features: None max_depth: None min_samples_leaf: 3 max_leaf_nodes: 925 n_estimators 100 666

Criterion: entropy max_features: log2 max_depth: 28 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 6581.180331693298
Criterion: gini max_features: None max_depth: 28 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 8485.908945619756
Criterion: entropy max_features: None max_depth: 28 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 6935.0604489625875
Criterion: gini max_features: auto max_depth: 81 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 7640.1229224387835
Criterion: entropy max_features: auto max_depth: 81 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 6581.180331693298
Criterion: gini max_features: log2 max_depth: 81 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 7640.1229224387835
Criterion: entropy max_features: log2 max_depth: 81 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 6581.180331693298
Criterion: gini max_features: None max_depth: 81 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 8485.908945

Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 76 max_leaf_nodes: 214 n_estimators 98 1384.2782652701521
Criterion: entropy max_features: log2 max_depth: None min_samples_leaf: 76 max_leaf_nodes: 214 n_estimators 98 1384.2782652701521
Criterion: gini max_features: None max_depth: None min_samples_leaf: 76 max_leaf_nodes: 214 n_estimators 98 1100.3355684749959
Criterion: entropy max_features: None max_depth: None min_samples_leaf: 76 max_leaf_nodes: 214 n_estimators 98 1100.3355684749959
Criterion: gini max_features: auto max_depth: 28 min_samples_leaf: 76 max_leaf_nodes: 214 n_estimators 98 1384.2782652701521
Criterion: entropy max_features: auto max_depth: 28 min_samples_leaf: 76 max_leaf_nodes: 214 n_estimators 98 1384.2782652701521
Criterion: gini max_features: log2 max_depth: 28 min_samples_leaf: 76 max_leaf_nodes: 214 n_estimators 98 1384.2782652701521
Criterion: entropy max_features: log2 max_depth: 28 min_samples_leaf: 76 max_leaf_nodes: 214 n_estimators 98

...

Criterion: gini max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: 507 n_estimators 65 9874.192233403948
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: 507 n_estimators 65 9515.339019624354
Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 1 max_leaf_nodes: 507 n_estimators 65 9874.192233403948
Criterion: entropy max_features: log2 max_depth: None min_samples_leaf: 1 max_leaf_nodes: 507 n_estimators 65 9515.339019624354
Criterion: gini max_features: None max_depth: None min_samples_leaf: 1 max_leaf_nodes: 507 n_estimators 65 8667.779905244204
Criterion: entropy max_features: None max_depth: None min_samples_leaf: 1 max_leaf_nodes: 507 n_estimators 65 7440.655783159818
Criterion: gini max_features: auto max_depth: 95 min_samples_leaf: 1 max_leaf_nodes: 507 n_estimators 65 9874.192233403948
Criterion: entropy max_features: auto max_depth: 95 min_samples_leaf: 1 max_leaf_nodes: 507 n_estimators 65 9515.33901

Criterion: gini max_features: auto max_depth: 68 min_samples_leaf: 1 max_leaf_nodes: None n_estimators 63 10997.133268724036
Criterion: entropy max_features: auto max_depth: 68 min_samples_leaf: 1 max_leaf_nodes: None n_estimators 63 10166.156407591816
Criterion: gini max_features: log2 max_depth: 68 min_samples_leaf: 1 max_leaf_nodes: None n_estimators 63 10997.133268724036
Criterion: entropy max_features: log2 max_depth: 68 min_samples_leaf: 1 max_leaf_nodes: None n_estimators 63 10166.156407591816
Criterion: gini max_features: None max_depth: 68 min_samples_leaf: 1 max_leaf_nodes: None n_estimators 63 7121.876963192858
Criterion: entropy max_features: None max_depth: 68 min_samples_leaf: 1 max_leaf_nodes: None n_estimators 63 6957.641291135433
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 3 max_leaf_nodes: None n_estimators 63 6940.024453216068
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 3 max_leaf_nodes: None n_estimators 63 6011.0

Criterion: gini max_features: auto max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 285 n_estimators 63 8023.433652454134
Criterion: entropy max_features: auto max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 285 n_estimators 63 6063.2112584006
Criterion: gini max_features: log2 max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 285 n_estimators 63 8023.433652454134
Criterion: entropy max_features: log2 max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 285 n_estimators 63 6063.2112584006
Criterion: gini max_features: None max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 285 n_estimators 63 8346.81934628679
Criterion: entropy max_features: None max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 285 n_estimators 63 6894.863077550891
Criterion: gini max_features: auto max_depth: 68 min_samples_leaf: 3 max_leaf_nodes: 285 n_estimators 63 8023.433652454134
Criterion: entropy max_features: auto max_depth: 68 min_samples_leaf: 3 max_leaf_nodes: 285 n_estimators 63 6063.2112584006
Criterion: 

Criterion: gini max_features: auto max_depth: None min_samples_leaf: 91 max_leaf_nodes: 507 n_estimators 63 1225.717011708898
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 91 max_leaf_nodes: 507 n_estimators 63 1225.717011708898
Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 91 max_leaf_nodes: 507 n_estimators 63 1225.717011708898
Criterion: entropy max_features: log2 max_depth: None min_samples_leaf: 91 max_leaf_nodes: 507 n_estimators 63 1225.717011708898
Criterion: gini max_features: None max_depth: None min_samples_leaf: 91 max_leaf_nodes: 507 n_estimators 63 767.09163223106
Criterion: entropy max_features: None max_depth: None min_samples_leaf: 91 max_leaf_nodes: 507 n_estimators 63 767.09163223106
Criterion: gini max_features: auto max_depth: 95 min_samples_leaf: 91 max_leaf_nodes: 507 n_estimators 63 1225.717011708898
Criterion: entropy max_features: auto max_depth: 95 min_samples_leaf: 91 max_leaf_nodes: 507 n_estimators 63 1225.7

Criterion: entropy max_features: None max_depth: 95 min_samples_leaf: 91 max_leaf_nodes: None n_estimators 67 561.09163223106
Criterion: gini max_features: auto max_depth: 68 min_samples_leaf: 91 max_leaf_nodes: None n_estimators 67 1372.717011708898
Criterion: entropy max_features: auto max_depth: 68 min_samples_leaf: 91 max_leaf_nodes: None n_estimators 67 1372.717011708898
Criterion: gini max_features: log2 max_depth: 68 min_samples_leaf: 91 max_leaf_nodes: None n_estimators 67 1372.717011708898
Criterion: entropy max_features: log2 max_depth: 68 min_samples_leaf: 91 max_leaf_nodes: None n_estimators 67 1372.717011708898
Criterion: gini max_features: None max_depth: 68 min_samples_leaf: 91 max_leaf_nodes: None n_estimators 67 561.09163223106
Criterion: entropy max_features: None max_depth: 68 min_samples_leaf: 91 max_leaf_nodes: None n_estimators 67 561.09163223106
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: 285 n_estimators 67 9860.1866311

In [ ]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

In [ ]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [ ]:
print(models)

In [ ]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

In [ ]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))